ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math


In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math



In [4]:
stocks = pd.read_csv('sp_500_stocks-2.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
496,YUM
497,ZBH
498,ZBRA
499,ZION


In [5]:
from secrets import IEX_CLOUD_API_TOKEN

In [6]:
symbol = 'AAPL'
api_url = f"https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 74783684, 'calculationPrice': 'close', 'change': 2.6, 'changePercent': 0.01922, 'close': 137.87, 'closeSource': 'official', 'closeTime': 1674248400884, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 137.805, 'delayedPriceTime': 1674248390160, 'extendedChange': 0.23, 'extendedChangePercent': 0.00167, 'extendedPrice': 138.1, 'extendedPriceTime': 1674262799323, 'high': 138.02, 'highSource': '15 minute delayed price', 'highTime': 1674248399976, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 137.89, 'iexCloseTime': 1674248399970, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 135.06, 'iexOpenTime': 1674225000988, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1674248399970, 'latestPrice': 137.87, 'latestSource': 'Close', 'latestTime': 'January 20, 2023', 'latestUpdate': 1674248400884, 'latestVolume': 80223626, 'low': 134.22, 'lowSource': '15 minute delayed 

In [7]:
price = data['latestPrice']
market_cap = data['marketCap']

In [8]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame( columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [9]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
     ],
    index = my_columns
    ),
    ignore_index=True



)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,137.87,2183338686310,N/A


In [10]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f"https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
            
        ],
        index = my_columns),
        ignore_index = True
    
    )

In [11]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,155.92,46163552477,N/A
1,AAL,16.34,10619380494,N/A
2,AAP,147.49,8866839070,N/A
3,AAPL,137.87,2183338686310,N/A
4,ABBV,149.59,264546999192,N/A
...,...,...,...,...
496,YUM,126.63,35670134472,N/A
497,ZBH,124.09,26040583323,N/A
498,ZBRA,297.03,15335598900,N/A
499,ZION,51.48,7702321976,N/A


In [12]:
#improve performance by chunking list
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [13]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
 #   print(symbol_groups[i])
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
                
            ],
            index = my_columns),
            ignore_index=True
        
        
        )
    

In [ ]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease try again: ")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

In [ ]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

#number_of_apple_shares = position_size/500
#print(math.floor(number_of_apple_shares))

In [ ]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        
        
    }

)

dollar_format = writer.book.add_format(
    {
        'mun_format': '$0.00'
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        
        
    }

)

integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        
        
    }

)

In [ ]:
writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
writer.save()

In [ ]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [ ]:
writer.save()